In [ ]:
# Task 2
import os
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist
from prettytable import PrettyTable

sample_input = []
sample_input.append("STIP/target_videos/cartwheel/Bodenturnen_2004_cartwheel_f_cm_np1_le_med_0.avi.txt")
sample_input.append("STIP/target_videos/sword_exercise/Blade_Of_Fury_-_Scene_1_sword_exercise_f_cm_np1_ri_med_3.avi.txt")
sample_input.append("STIP/target_videos/sword/AHF_longsword_against_Rapier_and_Dagger_Fight_sword_f_cm_np2_ri_bad_0.avi.txt")
sample_input.append("STIP/target_videos/drink/CastAway2_drink_u_cm_np1_le_goo_8.avi.txt")

column_labels = ["point-type", "x", "y", "t", "sigma2", "tau2", "detector-confidence"]
for i in range(1, 73):
    column_labels.append("dscr-hog (" + str(i) + ")")
for i in range(1, 91):
    column_labels.append("dscr-hof (" + str(i) + ")")

tau_values = [2, 4]
sigma_values = [4, 8, 16, 32, 64, 128]

with open('HoG_cluster_representatives.json', 'r') as fp:
    nt_centers_HOG = json.load(fp)
with open('HoF_cluster_representatives.json', 'r') as fp:
    nt_centers_HOF = json.load(fp)

In [ ]:
def find_closest_clusters(x, y):
    distances = euclidean_distances(x, y)
    return np.argmin(distances, axis=1)

In [ ]:
sample_bof_hog = []
sample_bof_hof = []

for video_name in sample_input:
    df = pd.read_csv(video_name, sep="\t", comment='#', header=None)
    df = df.nlargest(400, 6)

    df = df.dropna(axis=1)
    df.columns = column_labels

    hog_histograms = []
    hof_histograms = []

    for t in tau_values:
        for s in sigma_values:
    
            filtered_df = df[(df["tau2"] == t) & (df["sigma2"] == s)]
            if filtered_df.shape[0] != 0:
                hog_cluster_Id = find_closest_clusters(filtered_df.loc[:, "dscr-hog (1)":"dscr-hog (72)"], nt_centers_HOG[str((t, s))])
                hof_cluster_Id = find_closest_clusters(filtered_df.loc[:, "dscr-hof (1)":"dscr-hof (90)"], nt_centers_HOF[str((t, s))])
        
                hog_histogram, bin_edges = np.histogram(hog_cluster_Id, bins=np.arange(41))
                hof_histogram, bin_edges = np.histogram(hof_cluster_Id, bins=np.arange(41))
            elif filtered_df.shape[0] != 0:
                hog_histograms.append(numpy.zeros(480))
                hof_histograms.append(numpy.zeros(480))

            hog_histograms.append(hog_histogram)
            hof_histograms.append(hof_histogram)

    sample_bof_hog.append(np.concatenate(hog_histograms))
    sample_bof_hof.append(np.concatenate(hof_histograms))

In [ ]:
target_name = []
target_bof_hog = []
target_bof_hof = []

# Function to traverse the STIP directory
def process_STIPs_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            video_file = os.path.join(root, file)
            video_name = os.path.relpath(video_file, folder_path)

            with open(folder_path + "\\" + video_name, 'r') as fp:
                target_video = json.load(fp)

            target_name.append(video_name)
            target_bof_hog.append(target_video["hog"])
            target_bof_hof.append(target_video["hof"])

process_STIPs_in_folder("Task-4")

In [ ]:
hog_distances = cdist(sample_bof_hog, target_bof_hog, metric='euclidean')
hof_distances = cdist(sample_bof_hof, target_bof_hof, metric='euclidean')

In [ ]:
num_closest = 11
closest_hog_indices = []
closest_hof_indices = []

for i in range(len(sample_input)):
    # Get indices of the 10 closest data points
    indices_hog = np.argsort(hog_distances[i])[1:num_closest]
    closest_hog_indices.append(indices_hog)

    indices_hof = np.argsort(hof_distances[i])[1:num_closest]
    closest_hof_indices.append(indices_hof)

In [ ]:
print("HoG Closest Videos")
for i in range(len(sample_input)):
    print(sample_input[i])
    t = PrettyTable(["Rank", "Video Name", "Distance"])
    for j in range(10):
        index = closest_hog_indices[i][j]
        t.add_row([j + 1, (target_name[index])[:-5], hog_distances[i][index]])
    print(t)

In [ ]:
print("HoF Closest Videos")
for i in range(len(sample_input)):
    print(sample_input[i])
    t = PrettyTable(["Rank", "Video Name", "Distance"])
    for j in range(10):
        index = closest_hof_indices[i][j]
        t.add_row([j + 1, (target_name[index])[:-5], hof_distances[i][index]])
    print(t)